<a href="https://colab.research.google.com/github/alfia14/Covid-19_Image_Classifier/blob/main/Covid_19_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The heatmap generated shows high value of False Positives, and False Negatives, compared to the results obatined in the tutorial.
This maybe due to a decreased no of 'steps_per_epochs' used from 8 to 5.

In [ ]:
#Dataset : http://cb.lk/covid_19

!wget http://cb.lk/covid_19


--2023-10-22 05:15:22--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 104.21.72.203, 172.67.187.81, 2606:4700:3032::ac43:bb51, ...
Connecting to cb.lk (cb.lk)|104.21.72.203|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2023-10-22 05:15:23--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.72.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.72.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2023-10-22 05:15:23--  https://www.dropbox.com/s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca9e9

In [ ]:
  !unzip covid_19

Archive:  covid_19
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg  
  inflating: CovidDataset/Val/Covid/6b44464d-73a7-4cf3-bbb6-ffe7

In [ ]:
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [ ]:
 #CNN Based model in keras
 #its gng to be a layer based type layered architecture
 #helpful bcoz ???
 #will build multiple cnn layers and then classification layers
 #32 feature extract, ie will extract 32 diff type of filters in 1st layer
 #less no of filter in lower layers as they can detect less no of features
 #as u go deeper in n/w receptive filed increases, features it extracts gets large no of...
#kernel size in 3x3 ie standard choice

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu')) #adding 64 filters each of size 3x3
model.add(MaxPooling2D(pool_size=(2,2))) #default size is 2,2

model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam', metrics=['accuracy'])







# New Section

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                   

In [ ]:
#train frm scratch

#adam is default choice
#matrix is accuracy, ie we will do grad des optimization 4 adam, matrix on which we optimize model is gng 32 b class fic
#DEEPER INTO MODEL NEED 2 INCRS NO OF CHANNELS, will find more complexfeatures in the object frm distance?
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,

)
test_dataset = image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224,224), #224 standard size, image net samples also based on this
    batch_size = 32,
    class_mode = 'binary')


Found 224 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
#i/p shape saying what size we will feed in u
#bt then trget size is for shaping in2 trgt size


#similar for validation genr.
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')


Found 60 images belonging to 2 classes.


In [ ]:
#how to find label for final img- auto, 80-20 split btwn val & train

#far conversion- normalization

hist = model.fit(
    train_generator,
    steps_per_epoch=5,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)
#trainin now starting



Epoch 1/10
5/5 [==============================] - 62s 12s/step - loss: 0.8498 - accuracy: 0.6000 - val_loss: 0.6602 - val_accuracy: 0.8333
Epoch 2/10
5/5 [==============================] - 59s 12s/step - loss: 0.5273 - accuracy: 0.7750 - val_loss: 0.5553 - val_accuracy: 0.7000
Epoch 3/10
5/5 [==============================] - 61s 13s/step - loss: 0.4534 - accuracy: 0.8313 - val_loss: 0.3952 - val_accuracy: 0.9333
Epoch 4/10
5/5 [==============================] - 57s 11s/step - loss: 0.3895 - accuracy: 0.8625 - val_loss: 0.4370 - val_accuracy: 0.9667
Epoch 5/10
5/5 [==============================] - 58s 12s/step - loss: 0.3205 - accuracy: 0.8875 - val_loss: 0.3524 - val_accuracy: 0.9500
Epoch 6/10
5/5 [==============================] - 62s 12s/step - loss: 0.2600 - accuracy: 0.9125 - val_loss: 0.1483 - val_accuracy: 0.9667
Epoch 7/10
5/5 [==============================] - 62s 13s/step - loss: 0.1806 - accuracy: 0.9563 - val_loss: 0.1231 - val_accuracy: 0.9667
Epoch 8/10
5/5 [===========

In [ ]:
#grad map technique , class activation mapping,

In [ ]:
#Confusion matrix: gives u prediction vs normal tl=h br=h rest=0

In [ ]:
model.save("model_adv.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.evaluate(train_generator)

7/7 [==============================] - 24s 3s/step - loss: 0.1623 - accuracy: 0.9643


[0.16226020455360413, 0.9642857313156128]

In [ ]:
model.evaluate(validation_generator)

2/2 [==============================] - 6s 2s/step - loss: 0.1293 - accuracy: 0.9667


[0.1293080598115921, 0.9666666388511658]

In [ ]:
#above indicating low loss

In [ ]:
model = load_model('model_adv.h5')

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
%%writefile score.py


import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image, ImageOps
import cv2
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator



st.set_option('deprecation.showfileUploaderEncoding',False)
st.title("COVID-19 Classifier")

@st.cache(allow_output_mutation=True)
def load_model():
  model = tf.keras.models.load_model('/content/model_adv.h5')
  return model

with st.spinner('loading model...'):
  model = load_model()

file = st.file_uploader('UPLOAD IMAGE',type=["jpeg"])

def decode_img(image_data,model):
  size = (224,224)
  #image = ImageOps.fit(image_data,size, Image.ANTIALIAS)
  #img = np.asarray(image)
  #img_reshape = img[np.newaxis,...]
  img = tf.image.decode_img(image_data,channels=3)
  image = tf.image.resize(img,[224,224])
  img_arr = np.expand_dims(img_to_array(image), axis=0)
  prediction = model.predict(img_arr)
  return prediction
if file is None:
    st.text("UPLOAD THE IMAGE")
else:
    image = Image.open(file).convert("RGB")
    st.image(image, caption='CLASSIFYING THE IMAGE', use_column_width=True)
    p = decode_img(image,model)

    classes=['COVID','NORMAL']
    string ="PREDICTED CLASS IS " + classes[np.argmax(p)]
    st.success(string)




Writing score.py


In [ ]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [ ]:
!pip install pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=bdedb972e79d9879da36efc7fc52ca9536731d974621ec739c171253bd06ed1c
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [ ]:
!ngrok authtoken 1pRNj9UUQIxPaF8Wmtc5WUpzQrw_7ZDADCzAHtPywg2gBrFKt

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
url = ngrok.connect(port=8501)
url


'http://0b3f-34-138-204-138.ngrok-free.app'

In [ ]:
!cat nohup.out




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.204.138:8501

